In [1]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.optim as optim
from torchvision import transforms, models

In [2]:
def load_image(img_path, max_size=400, shape=None):
    image = Image.open(img_path).convert('RGB')  
  
    if max(image.size) > max_size:
        size = max_size
    else:
        size = max(image.size)
	
    if shape is not None:
        size = shape
  
    in_transform = transforms.Compose([transforms.Resize((size, int(1.5*size))),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
  
    image = in_transform(image)[:3, :, :].unsqueeze(0)
  
    return image

In [9]:
style = load_image("chicken.png")
style.shape

torch.Size([1, 3, 400, 600])

In [10]:
def im_convert(tensor):
    image = tensor.to("cpu").clone().detach()
    image = image.numpy().squeeze()
    image = image.transpose(1, 2, 0)
    image = image * np.array((0.229, 0.224, 0.225)) + np.array((0.485, 0.456, 0.406))
    image = image.clip(0, 1)
  
    return image

In [11]:
def get_features(image, model, layers=None):
    if layers is None:
        layers = {'0': 'conv1_1','5': 'conv2_1',
              '10': 'conv3_1',
              '19': 'conv4_1',
              '21': 'conv4_2',  ## content layer
              '28': 'conv5_1'}
    features = {}
    x = image
    for name, layer in enumerate(model.features):
        x = layer(x)
        if str(name) in layers:
            features[layers[str(name)]] = x
  
    return features

In [12]:
def gram_matrix(tensor):
    _, n_filters, h, w = tensor.size()
    tensor = tensor.view(n_filters, h * w)
    gram = torch.mm(tensor, tensor.t())
  
    return gram

In [13]:
torch.utils.model_zoo.load_url('https://download.pytorch.org/models/vgg19-dcbb9e9d.pth', model_dir='/results/')

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /results/vgg19-dcbb9e9d.pth
  1%|▍                                                                 | 4284416/574673361 [00:11<24:54, 381613.27it/s]


KeyboardInterrupt: 

In [14]:
vgg = models.vgg19()
vgg.load_state_dict(torch.load("style.png"))

for param in vgg.parameters():
    param.requires_grad_(False)

UnpicklingError: A load persistent id instruction was encountered,
but no persistent_load function was specified.